<table style="background-color: transparent;">
<tr>
<td><img src="../tutorial/spark-logo-trademark.png" width="250px" /></td>
<td> + </td>
<td><img src="../tutorial/Python_logo-768x325.png" width="350px" /></td>
<td> + </td>
<td><img src="../tutorial/logo-gray.png" width="250px" /></td>
</tr>
</table>

# Calcular o valor numérico de $\pi$ no cluster

Quando escolhemos como _kernel_ do _notebook_ o `PySpark`, o código é executado localmente na máquina principal do cluster. Para executar nos diferentes nós, é necessário usar o comando `spark-submit`, e o código deve estar todo dentro de um único ficheiro `.py`.

Ao ser executado o código deste ficheiro, a variável `sc` não está imediatamente disponível. Para isso é necessário acrescentar ao código as seguintes instruções:

    import pyspark
    conf = pyspark.SparkConf()
    sc = pyspark.SparkContext(conf=conf)
    
A função  `SparkConf()` permite definir opções para a forma como o código é executado no cluster.

## Verificar se existe o spark context

**Recorde que no cluster apenas é possível um kernel PySpark activo a cada instante.**

In [ ]:
sc

## Imports e definição de funções

In [ ]:
from scipy import random
import time

In [ ]:
def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1

## Cálculo do PI

A variável `NUM_SAMPLES` define o número de amostras no cálculo do $\pi$.

In [ ]:
NUM_SAMPLES = 100000000

In [ ]:
t0 = time.time()
count = sc.parallelize(range(0, NUM_SAMPLES)) \
             .filter(inside).count()
t1 = time.time()

print ("Pi é aproximadamente %f" % (4.0 * count / NUM_SAMPLES))
print ("O cálculo demorou %.2fs localmente." % (t1-t0))

## Criar um ficheiro python para correr no cluster

**Recorde:** o _magic_ `%%file` cria um ficheiro com o conteúdo da célula do notebook. Assim, a próxima linha cria um ficheiro chamado compute_pi.py, com o conteúdo:

    from scipy import random
    import pyspark
    ...
    print ("Pi é aproximadamente %f" % (4.0 * count / NUM_SAMPLES))
    print ("O cálculo demorou %.2fs localmente." % (t1-t0))

In [ ]:
%%file compute_pi.py
from scipy import random
import pyspark
import time
def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1
NUM_SAMPLES = 100000000
conf = pyspark.SparkConf()
sc = pyspark.SparkContext(conf=conf)
t0 = time.time()
count = sc.parallelize(range(0, NUM_SAMPLES)) \
             .filter(inside).count()
t1 = time.time()
print ("Pi é aproximadamente %f" % (4.0 * count / NUM_SAMPLES))
print ("O cálculo demorou %.2fs." % (t1-t0))

### Verificar o conteúdo do ficheiro `compute_pi.py`:

In [ ]:
!cat compute_pi.py

## Executar o ficheiro `compute_pi.py` no cluster:

In [ ]:
!/usr/share/spark/bin/spark-submit \
--master spark://192.168.1.105:7077 \
compute_pi.py